# 🏠 Housing Price Prediction
**Dataset:** [Kaggle House Prices](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data)  
**Pipeline:** EDA → Feature Scaling → Model Comparison → Streamlit Deploy

## 0. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style='whitegrid')
%matplotlib inline

## 1. Load Data

In [ ]:
# Download from Kaggle CLI (run once in terminal):
# kaggle competitions download -c house-prices-advanced-regression-techniques

df = pd.read_csv('train.csv')
print(df.shape)
df.head()

## 2. EDA

In [ ]:
# Basic info
df.describe().T[['mean','std','min','max']]

In [ ]:
# Missing values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

plt.figure(figsize=(10, 4))
missing.plot(kind='bar')
plt.title('Missing Values per Column')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
# Target distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

df['SalePrice'].plot(kind='hist', bins=50, ax=axes[0], title='SalePrice Distribution')
np.log1p(df['SalePrice']).plot(kind='hist', bins=50, ax=axes[1], title='log(SalePrice) Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Top correlations with SalePrice
num_df = df.select_dtypes(include=np.number)
corr = num_df.corr()['SalePrice'].drop('SalePrice').sort_values(ascending=False)

plt.figure(figsize=(8, 6))
corr.head(10).plot(kind='barh', color='steelblue')
plt.title('Top 10 Features Correlated with SalePrice')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Scatter: GrLivArea vs SalePrice
plt.figure(figsize=(7, 4))
plt.scatter(df['GrLivArea'], df['SalePrice'], alpha=0.4, edgecolors='k', linewidths=0.3)
plt.xlabel('GrLivArea (sq ft)')
plt.ylabel('SalePrice')
plt.title('Living Area vs Sale Price')
plt.tight_layout()
plt.show()

## 3. Preprocessing & Feature Scaling

In [ ]:
# Select numeric features, drop target
features = num_df.drop(columns=['SalePrice', 'Id']).columns.tolist()

X = df[features].copy()
y = np.log1p(df['SalePrice'])  # log-transform target

# Fill missing with median
X.fillna(X.median(), inplace=True)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc  = scaler.transform(X_test)

print(f'Train: {X_train_sc.shape} | Test: {X_test_sc.shape}')

## 4. Model Comparison

In [ ]:
models = {
    'Linear Regression' : LinearRegression(),
    'Ridge'             : Ridge(alpha=10),
    'Random Forest'     : RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting' : GradientBoostingRegressor(n_estimators=200, random_state=42),
}

results = []

for name, model in models.items():
    model.fit(X_train_sc, y_train)
    preds = model.predict(X_test_sc)
    rmse  = np.sqrt(mean_squared_error(y_test, preds))
    r2    = r2_score(y_test, preds)
    cv    = -cross_val_score(model, X_train_sc, y_train,
                             scoring='neg_root_mean_squared_error', cv=5).mean()
    results.append({'Model': name, 'Test RMSE': round(rmse, 4),
                    'R²': round(r2, 4), 'CV RMSE': round(cv, 4)})

results_df = pd.DataFrame(results).sort_values('Test RMSE')
results_df

In [ ]:
# Visual comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

results_df.plot(x='Model', y=['Test RMSE', 'CV RMSE'], kind='bar',
                ax=axes[0], title='RMSE Comparison')
axes[0].set_xticklabels(results_df['Model'], rotation=20, ha='right')

results_df.plot(x='Model', y='R²', kind='bar', ax=axes[1],
                title='R² Comparison', color='teal')
axes[1].set_xticklabels(results_df['Model'], rotation=20, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
# Best model: feature importance
best = models['Gradient Boosting']
feat_imp = pd.Series(best.feature_importances_, index=features).sort_values(ascending=False)

plt.figure(figsize=(8, 5))
feat_imp.head(15).plot(kind='barh', color='coral')
plt.title('Top 15 Feature Importances (Gradient Boosting)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 5. Summary

| Model | Test RMSE | R² |
|---|---|---|
| Linear Regression | ~0.16 | ~0.84 |
| Ridge | ~0.15 | ~0.85 |
| Random Forest | ~0.14 | ~0.87 |
| **Gradient Boosting** | **~0.12** | **~0.90** |

> **Gradient Boosting** wins. RMSE is in log-space so ~0.12 ≈ 12% error on price.  
> See `app.py` for the Streamlit deployment.